In [21]:
from langchain_community.document_loaders import CSVLoader

In [22]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
DATA_DIR = "couresfj.csv"

In [23]:
def load_docs(directory):
    
    loader = CSVLoader(directory)
    documents = loader.load()
    return documents

loaded_documents = load_docs(DATA_DIR)

In [24]:
loaded_documents

[Document(page_content='CourseDescription: CourseTitle: Python for beginners\n CourseDescription\n Master the Features of Python Language\n Install Python Virtual Machine and the Eclipse IDE(PyDev)\n Execute your first python program\n Learn various simple types as well as collection types\n Define logic using conditional statements ,looping constructs\n Use the different types of operators\n See the input and output functions in action\n Pass Command line arguments\n Create and use functions , Lambdas Decorators and Generators\n Learn what Object Oriented Programming is the four OOPs principles\n InstructorName: Bharath Thippireddy\n Price: $79', metadata={'source': 'couresfj.csv', 'row': 0}),
 Document(page_content="CourseDescription: Learn Python Programming Masterclass\n CourseDescription:\n Have a fundamental understanding of the Python programming language.\n Have the skills and understanding of Python to confidently apply for Python programming jobs.\n Acquire the pre-requisite 

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=20)
documents=text_splitter.split_documents(loaded_documents)
documents[:5]

[Document(page_content='CourseDescription: CourseTitle: Python for beginners\n CourseDescription\n Master the Features of Python Language\n Install Python Virtual Machine and the Eclipse IDE(PyDev)\n Execute your first python program\n Learn various simple types as well as collection types\n Define logic using conditional statements ,looping constructs\n Use the different types of operators\n See the input and output functions in action\n Pass Command line arguments\n Create and use functions , Lambdas Decorators and Generators\n Learn what Object Oriented Programming is the four OOPs principles\n InstructorName: Bharath Thippireddy\n Price: $79', metadata={'source': 'couresfj.csv', 'row': 0}),
 Document(page_content="CourseDescription: Learn Python Programming Masterclass\n CourseDescription:\n Have a fundamental understanding of the Python programming language.\n Have the skills and understanding of Python to confidently apply for Python programming jobs.\n Acquire the pre-requisite 

In [26]:
documents

[Document(page_content='CourseDescription: CourseTitle: Python for beginners\n CourseDescription\n Master the Features of Python Language\n Install Python Virtual Machine and the Eclipse IDE(PyDev)\n Execute your first python program\n Learn various simple types as well as collection types\n Define logic using conditional statements ,looping constructs\n Use the different types of operators\n See the input and output functions in action\n Pass Command line arguments\n Create and use functions , Lambdas Decorators and Generators\n Learn what Object Oriented Programming is the four OOPs principles\n InstructorName: Bharath Thippireddy\n Price: $79', metadata={'source': 'couresfj.csv', 'row': 0}),
 Document(page_content="CourseDescription: Learn Python Programming Masterclass\n CourseDescription:\n Have a fundamental understanding of the Python programming language.\n Have the skills and understanding of Python to confidently apply for Python programming jobs.\n Acquire the pre-requisite 

In [27]:
## Vector Embedding And Vector Store
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

embedding = OpenAIEmbeddings()

In [28]:

db = Chroma.from_documents(documents, embedding, persist_directory="./../vectordb")
db.persist()

In [29]:
query = "cho tôi 1 khóa học về data"
retireved_results=db.similarity_search(query)
print(retireved_results[0].page_content)

CourseDescription: The Data Science Course: Complete Data Science Bootcamp
 Complete Data Science Training: Mathematics, Statistics, Python, Advanced Statistics in Python, Machine & Deep Learning. ???c t?o b?i , 365 Careers Team. The course provides the entire toolbox you need to become a data scientist
 Fill up your resume with in demand data science skills: Statistical analysis, Python programming with NumPy, pandas, matplotlib, and Seaborn, Advanced statistical analysis, Tableau, Machine Learning with stats models and scikit-learn, Deep learning with TensorFlow
 Impress interviewers by showing an understanding of the data science field
 Learn how to pre-process data
 Understand the mathematics behind Machine Learning (an absolute must which other courses don’t teach!)
 Start coding in Python and learn how to use it for statistical analysis
 Perform linear and logistic regressions in Python
 Carry out cluster and factor analysis


In [35]:
db2 = Chroma(persist_directory="./../vectordb", embedding_function=embedding)

In [38]:
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.chat_models.openai import ChatOpenAI

qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-3.5-turbo"),
    chain_type="stuff",
    retriever=db2.as_retriever(search_type="mmr", search_kwargs={"k": 5}),
)

In [39]:
query = "tìm giúp tôi 1 khóa học về Data"
qa.run(query)

C:\Users\txkie\AppData\Roaming\Python\Python310\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'Dựa vào những mô tả khóa học về Data Science ở trên, có thể bạn quan tâm đến khóa học "The Data Science Course: Complete Data Science Bootcamp". Đây là khóa học cung cấp một loạt các kỹ năng từ toán học, thống kê, Python, Machine Learning, Deep Learning và nhiều kỹ năng khác cần thiết để trở thành một Data Scientist.'